# Import libs and define functions

In [39]:
import pandas as pd
import subprocess
import os
import shutil
import re
import sys 

In [40]:
def get_columns_to_pause(df):
    pattern = re.compile(r'^pause\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [41]:
def create_silent_audio(duration, output_file):
    # Command to create silent audio with ffmpeg
    command = [
        'ffmpeg',
        '-f', 'lavfi',
        '-i', 'anullsrc=channel_layout=stereo:sample_rate=48000',
        '-t', duration,
        '-c:a', 'aac',
        '-b:a', '191k',
        output_file
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Silent audio file {output_file} created successfully.")
        
    except subprocess.CalledProcessError as e:
        print(f"Error creating silent audio file: {e}")

In [42]:
def remove_end_in_name(filename):
    splitted = filename.split("_")
    splitted.pop()
    joined_string = "_".join(splitted)
    return joined_string

In [43]:
def get_creation_time(file_path):
    try:
        creation_time = os.path.getctime(file_path)
        return pd.to_datetime(creation_time, unit='s')
    except Exception as e:
        print(f"Error getting creation time for {file_path}: {e}")
        return None

In [44]:
def get_the_table_data():
    for f in os.listdir():
        if f.endswith(".csv"):
                video_schnitt_df = pd.read_csv(f)
                print(f"Table {f} loaded succesfully")
                return video_schnitt_df
    raise ValueError("There is no csv in this directory")

In [45]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [46]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [47]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c:v','libx264', #libx264 to encode and copy to test
        '-c:a', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [48]:
def create_folder(folder_name):
    try:
        # Create a new folder in the current working directory
        os.mkdir(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    except FileExistsError:
        print(f"Folder '{folder_name}' already exists.")

In [49]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'libx264', output_file # libx264 to encode and copy to test (but that didnt work the last time)
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [50]:
def get_audio(input_file,output_file,cut_head,cut_tail):
    
    #Construct command
    command = ['ffmpeg',
               '-i', input_file,
               '-ss', cut_head,
               '-to', cut_tail,
               '-vn',
               '-c:a', 'copy', output_file]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [51]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'libx264', # needs to stay libx264
               '-c:a', 'copy', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [52]:
def get_columns_to_cut(df):
    pattern = re.compile(r'^rausschneiden\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [53]:
def get_list_of_evens(input_list):
    dummy = []
    for i in range(len(input_list)):
        if i % 2 == 0:
            dummy.append(i)
    return dummy

In [54]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file, # txt file
         '-c:v', 'copy', #libx264 to encode if needed
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

In [55]:
def check_for_non_zero_digit(string):
    # Define a regular expression pattern to match any digit other than 0
    pattern = re.compile('[1-9]')

    if pattern.search(string):
        return True       
    else:
        return False


In [56]:
def extract_data_from_row(df,row_index,columns):
    dic = df.loc[row_index,columns].to_dict()

    return dic

In [57]:
def check_number_of_columns(df):
    columns = get_columns_to_cut(df)
    for row, video_name in df["dateiname"].items():
        data = df["rausschneiden1_bis"][row]
        
        if data != "nan":
            dic = extract_data_from_row(df,row,columns)
            counter = 0

            for key, value in dic.items():
                
                if (key != "rausschneiden1_ab"
                        and value == "nan"):
                    counter += 1
            
            if counter < 3:
                error_message = (
                    f"Error: Insufficient 'rausschneiden...' columns in row of Normalverteilung.mp4.\n"
                    "At least 3 free 'rausschneiden...' columns at the end are required to execute the code.\n"
                    "Please update the CSV file accordingly and restart the script."
                )
                
                print(error_message)
                
                sys.exit("Script terminated due to error.")



In [58]:
# def check_special_case(df):
#     columns = get_columns_to_cut(df)
#     for idx, video_name in df["dateiname"].items():
#         data = df["rausschneiden1_ab"][idx]
#         if check_for_non_zero_digit(data) == True:
#             dic = extract_data_from_row(df,idx,columns)
#             counter = 0
#             for key,value in dic.items():
#                 if value == "nan":
#                     counter += 1
#             if counter < 2:
#                 print(f"""Error: Insufficient 'rausschneiden...' columns in row of {video_name}.\n
#                       "At least 2 free 'rausschneiden...' columns at the end are required to execute the code.\n"
#                       "Please update the CSV file accordingly and restart the script.""")
                
#                 sys.exit("Script terminated due to error.")

In [59]:
def solve_special_case(df):
    
    for idx,video_name in df["dateiname"].items():
        
        if check_for_non_zero_digit(df["rausschneiden1_ab"][idx]) == False:
            continue

        else:
            columns = get_columns_to_cut(df)
            dic = extract_data_from_row(df,idx,columns)
            dummy = []
            counter = 0

            # put values from row into list and change first two columns to 000000
            for key, value in dic.items():
                if value != "nan":
                    dummy.append(value)
                if counter < 2:
                    dic[key] = "00:00:00"
                    counter += 1
        # fülle das jeweilige dic solange mit daten bis liste leer
        for key, value in dic.items():
            if value != "00:00:00" and dummy:
                dic[key] = dummy.pop(0)
        
        for key, value in dic.items():
            df.loc[idx, key] = value
    
    return df

In [60]:
def convert_timestamp_to_right_format(input):
    if input != "nan":
        input = pd.to_timedelta(input)
        dummy_date = pd.Timestamp('1900-01-01')
        new_timestamp = dummy_date + input
        new_timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
        return str(new_timestamp)

In [61]:
def get_columns_to_divide(df):
    pattern = re.compile(r'^teilen\d+_[a-z]+$')
    dummy = []
    for column in df.columns:
        if pattern.match(column):
            dummy.append(column)
    return dummy

In [62]:
def calculate_time(*timestamps,kind="addition"):
    """
    Calculate time based on the given timestamps and operation type.

    Parameters:
    - timestamps (str): Variable number of timestamps.
    - kind (str): The operation type. Allowed values are 'addition' and 'subtraction'.
    """
    
    #addition
    if kind == "addition":
        stamp1 = None
        for timestamp in timestamps:
            if timestamp == "nan":
                timestamp = "00:00:00"
        
        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 + pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                return timestamp
    
    #subtraction
    elif kind == "subtraction":
        stamp1 = None
        for timestamp in timestamps:
            if timestamp == "nan":
                timestamp = "00:00:00"        

        # calculate timestamp
            if stamp1 is None:
                stamp1 = timestamp

            else:
                timedelta = stamp1 - pd.to_timedelta(timestamp)
                dummy_date = pd.Timestamp('1900-01-01')
                new_timestamp = dummy_date + timedelta
                new_timestamp = new_timestamp.strftime('%H:%M:%S.%f')[:-3]
                if new_timestamp == '00:00:00.000':
                    new_timestamp = stamp1
                return new_timestamp        

In [63]:
calculate_time("00:01:00","00:01:00", kind="subtraction")

'00:01:00'

In [64]:
def cut_in_2_pieces(input_file, timestamp, head_output, tail_output):
    
    # -ss = ab dort; -to = bis dort
    head_cmd = ['ffmpeg', '-i', input_file, '-to', timestamp, '-c:v', 'copy', '-c:a', 'copy', head_output] # "-c:v" -> "libx264" to encrypt or "copy" to test
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', timestamp, '-c:v', 'copy', '-c:a', 'copy', tail_output] # "-c:v" -> "libx264" to encrypt or "copy" to test

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [65]:
def calculate_timestamp_from_pauses(input_timestamp,row_number):
    columns_to_pause = get_columns_to_pause(video_schnitt_df)
    pattern = re.compile(r'^pause\d+_bei$')
    result = "00:00:00"
    input_timestamp = convert_timestamp_to_right_format(input_timestamp)


    for col_num,col in enumerate(video_schnitt_df[columns_to_pause]):
        
        ab_timestamp = video_schnitt_df[columns_to_pause].iloc[row_number, col_num]
        ab_timestamp = convert_timestamp_to_right_format(ab_timestamp)
        
        try:
            dauer_timestamp = video_schnitt_df[columns_to_pause].iloc[row_number, col_num+1]
        except Exception as e:
            print(e)

        # check wether the timestamp in 'rausschneidenX_bis' is smaller than input
        if pattern.match(col):
            if str(ab_timestamp) < str(input_timestamp):
                # print(video_name, "col name:", col)
                # print("ab timestamp:", ab_timestamp, "ist kleiner als input Tiestamp:",input_timestamp)
                # print("dauer zum adden ist:", dauer_timestamp)
                result = calculate_time(result,dauer_timestamp,"addition")
                # print("Summe:",result)
                
    new_timestamp = calculate_time(input_timestamp,result,kind="addition")
    # print("alt:", input_timestamp,"plus:", result, "neuer Teimestamp:",new_timestamp)
    result = "00:00:00"

    return new_timestamp


In [66]:
def calculate_timestamp(input_timestamp,row_number):
    columns_to_cut = get_columns_to_cut(video_schnitt_df)
    pattern = re.compile(r'^rausschneiden\d+_bis$')
    result = "00:00:00"
    input_timestamp = convert_timestamp_to_right_format(input_timestamp)

    for col_num,col in enumerate(video_schnitt_df[columns_to_cut]):
        
        ab_timestamp = video_schnitt_df[columns_to_cut].iloc[row_number, col_num-1]
        bis_timestamp = video_schnitt_df[col][row_number]
        bis_timestamp = convert_timestamp_to_right_format(bis_timestamp)

        # check wether the timestamp in 'rausschneidenX_bis' is smaller than input
        if pattern.match(col):
            if bis_timestamp != "nan" and str(bis_timestamp) < str(input_timestamp):
                # print(video_name, "col name:", col)
                # print("bis timestamp:", bis_timestamp, "ist kleiner als input Tiestamp:",input_timestamp)
                # print("ab Timestamp ist:", ab_timestamp)
                # print("bis Timestamp:",bis_timestamp,"minus",ab_timestamp)
                diff = calculate_time(bis_timestamp,ab_timestamp,kind="subtraction")
                result = calculate_time(result,diff,kind="addition")
                # print("Differenz:",diff,"Summe:",result)
                
    new_timestamp = calculate_time(input_timestamp,result,kind="subtraction")
    # print("neuer Teimestamp:",new_timestamp)
    result = "00:00:00"

    return new_timestamp


In [67]:
def remove_parts(video_name):
    original_prefix = video_name.split('.')[0]
    pattern = re.compile(rf'^{re.escape(original_prefix)}_part\d+.*$')

    for f in os.listdir():
        if f.endswith(".mp4") and pattern.match(f):
            os.remove(f)

In [68]:
def rename_part1_videos():
    """This function is a quick solution to solve following problem:
    If a video just needs one cut, and its no special case, then there will be 3 parts and just one will be odd. 
    That means there is no concatted, and that means the part1 video needs to be moved"""

    for video_name in video_schnitt_df["dateiname"]:
        original_prefix = video_name.split('.')[0]
        suffix = "_part1.mp4"
        pattern = re.compile(rf'^{re.escape(original_prefix)}_part\d+.*$')
        counter = 0
        #print("new loop")

        for f in os.listdir():
            if f.endswith(".mp4") and pattern.match(f):
                counter += 1
                if counter > 1:
                    #print("video name:", video_name, "prefix:", original_prefix)
                    #print("file to remove:", original_prefix + suffix, "endfile:", original_prefix + '_concatted1.mp4')
                    os.rename(original_prefix + suffix, original_prefix + "_concatted1.mp4")
                    break

In [69]:
# def convert_to_time(input_string):
#     try:
#         time_type = pd.to_datetime(input_string, format='%H:%M:%S.%f').dt.time
        
#         # Get the format
#         formatted_time = time_type.dt.strftime('%H:%M:%S.%f')
#         return formatted_time
#     except ValueError:
#         print("Input string is not in the correct format.")
#         return None

# Get file and clean it

In [129]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)
                    

Table Videoschnitt_Liste - Spaltennamen (1).csv loaded succesfully
Table Columns cleaned


# Check Special Cases and input error

In [130]:
check_number_of_columns(video_schnitt_df)

In [131]:
solve_special_case(video_schnitt_df)

,dateiname,notiz,rausschneiden1_ab,rausschneiden1_bis,rausschneiden2_ab,rausschneiden2_bis,rausschneiden3_ab,rausschneiden3_bis,rausschneiden4_ab,standbild_bis,teilen1_bei,teilen2_bei,teilen3_bei,teilen4_bei,pause1_bei,pause1_dauer,pause2_bei,pause2_dauer,pause3_bei,pause3_dauer
0,Normalverteilung.mp4,nan,nan,nan,nan,nan,nan,nan,nan,nan,00:01:30,00:03:00,nan,nan,00:00:30,00:00:10,00:01:00,00:00:30,00:02:00,00:01:00
1,ELR_Geradengleichung.mp4,nan,nan,nan,nan,nan,nan,nan,nan,nan,00:01:30,nan,nan,nan,00:01:00,00:00:10,00:02:00,00:00:30,nan,nan
2,ELR_Geradengleichung_Einleitung.mp4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,00:02:00,00:00:30,nan,nan,nan,nan
3,Normalverteilung2.mp4,nan,nan,nan,nan,nan,nan,nan,nan,nan,00:01:30,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


# Get Standbild

In [73]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        else:
            cut_head = video_schnitt_df["standbild_bis"][idx]
        
        output_file = f"{video_name.split('.')[0]}_standbild.mp4"

        print("video_name:",video_name,"cuttime:",cut_head,)
        get_standbild(video_name,output_file,cut_head)
except Exception as error:
    print("An error occurred:", error)

# Get audio

In [74]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue

        #Define arguments
        output_file = f"{video_name.split('.')[0]}_head_audio.mp4"
        cut_head = "00:00:00"
        cut_tail = video_schnitt_df["standbild_bis"][idx]

        print("video_name",video_name,"output_file", output_file,"cut_head",cut_head,"cut_tail:", cut_tail)
        get_audio(video_name,output_file,cut_head,cut_tail)

except Exception as error:
    print("An error occurred:", error)

# Cut head of the original Video

In [75]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        # define arguments
        cut_head = video_schnitt_df["standbild_bis"][idx]
        cut_tail = get_video_duration(video_name)
        output_file = f"{video_name.split('.')[0]}_ohne_start_standbild.mp4"
    
        print("input name:",video_name,"Schnittpunkte:", cut_head, cut_tail)
        cut_head_tail(video_name, output_file, cut_head, cut_tail)
except Exception as error:
    print("An error occurred:", error)

# Merge audio and video

In [76]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        video_file = f"{video_name.split('.')[0]}_standbild.mp4"
        audio_file = f"{video_name.split('.')[0]}_head_audio.mp4"
        output_file = f"{video_name.split('.')[0]}_merged_start.mp4"
        
        print("video:", video_file, "audio:", audio_file, "output", output_file)
        merge_audio_and_video(video_file,audio_file,output_file)

except Exception as error:
    print("An error occured:", error)

# Concat to endproduct

In [77]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if video_schnitt_df["standbild_bis"][idx] == "nan":
            continue
        
        #define input for the text file with the videos we want to concat
        video1 = f"{video_name.split('.')[0]}_merged_start.mp4"
        video2 = f"{video_name.split('.')[0]}_ohne_start_standbild.mp4"
        textfile_content = f"file '{video1}'\nfile '{video2}'"

        #create the file with content
        with open('dummy.txt', 'w') as textfile:
            textfile.write(textfile_content)

        #define parameters for concat function
        output_file = f"{video_name.split('.')[0]}_standbild_endprodukt.mp4"
        textfile_name = "dummy.txt"
        print("file content:", textfile_content)
        concatenate_videos(textfile_name,output_file)
        
except Exception as error:
    print("An error occured:", error)

# Create folder and move files

In [78]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

Folder 'endprodukte' created successfully.
Folder 'script_output' created successfully.


In [79]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [80]:
for filename in os.listdir():
    if filename.endswith("endprodukt.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

# Start to cut things out

In [81]:
# cut all the videos that did not use the standbild ersetzen script
try:
    # videos need to be in same directory with python script
    trash_list = []
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if not os.path.exists(video_name):
            print("Following file does not exist:",video_name)
            continue

        #check ob video schon standbid ersetzt bekommen, use other videoname
        if video_schnitt_df["standbild_bis"][idx] != "nan":
            print(video_name,"skipped because other video in endprodukte needs to be used")
            continue

        columns_to_cut = get_columns_to_cut(video_schnitt_df)
        evens = get_list_of_evens(columns_to_cut)

        parts_list = []
        video1 = 0
        video2 = 0
        end_reached = None
        print("Video Name to cut:", video_name)

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]
                print("index:",i,"Column:",col)
                print("Timestamps before changing:",cut_head,cut_tail)

                #logic to handle the nans 
                
                if cut_head != "nan" and cut_head != "00:00:00" and i == 0:
                    print("Skipped because of special case")
                    
                    break
                
                if cut_head == "nan" and cut_tail == "nan":
                    print("Break used")
                    break
                
                if end_reached == True:
                    break

                if  cut_head == "nan":
                    cut_head = "00:00:00"
                    print("cut_head ersetzt")

                if cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)
                    end_reached = True
                    print("end reached")
                    
                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("Timestamps after changing:",cut_head,cut_tail)
                cut_head_tail(video_name, output_file, cut_head, cut_tail)
                
                if i not in evens:
                    parts_list.append(output_file)
                else:
                    trash_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    concatenate_videos(textfile_name,output_file)    
                    remove_parts(video_name)            

        except Exception as e:
            print("skip to next video because:",e)

except Exception as error:
    print("An error occured:", error)   

Video Name to cut: Normalverteilung.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan nan
Break used
Video Name to cut: ELR_Geradengleichung.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan nan
Break used
Video Name to cut: ELR_Geradengleichung_Einleitung.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan nan
Break used
Video Name to cut: Normalverteilung2.mp4
index: 0 Column: rausschneiden1_ab
Timestamps before changing: nan nan
Break used
Following file does not exist: nan


## cut all the videos that did not use the standbild ersetzen script


In [82]:

try:
    # videos need to be in same directory with python script
    trash_list = []
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if not os.path.exists(video_name):
            print("Following file does not exist:",video_name)
            continue

        #check ob video schon standbid ersetzt bekommen, use other videoname
        if video_schnitt_df["standbild_bis"][idx] == "nan":
            print(f"skipped this {video_name} because it is already finished")
            continue
        
        folder_name = "endprodukte/"
        new_video_name = folder_name + f"{video_name.split('.')[0]}_standbild_endprodukt.mp4"

        columns_to_cut = get_columns_to_cut(video_schnitt_df)
        evens = get_list_of_evens(columns_to_cut)

        parts_list = []
        video1 = 0
        video2 = 0
        end_reached = None
        print("Video Name to cut:", video_name, "actual input to cut:", new_video_name)

        try:
            for i,col in enumerate(columns_to_cut):
                cut_head = video_schnitt_df[col][idx]
                cut_tail = video_schnitt_df[columns_to_cut[i+1]][idx]
                print("index:",i,"Column:",col)
                print("Timestamps before changing:",cut_head,cut_tail)

                #logic to handle the nans
                if cut_head != "nan" and cut_head != "00:00:00" and i == 0:
                    print("Skipped because of special case")
                    break
                
                if cut_head == "nan" and cut_tail == "nan":
                    print("Break used")
                    break
                
                if end_reached == True:
                    print("Break used")
                    break

                if  cut_head == "nan":
                    cut_head = "00:00:00"
                    print("cut_head ersetzt")

                if cut_tail == "nan":
                    cut_tail = get_video_duration(video_name)
                    end_reached = True
                    print("end reached")
                    
                output_file = f"{video_name.split('.')[0]}_part{i}.mp4"
                print("Timestamps after changing:",cut_head,cut_tail)
                cut_head_tail(new_video_name, output_file, cut_head, cut_tail)
                
                if i not in evens:
                    parts_list.append(output_file)
                else:
                    trash_list.append(output_file)

                if len(parts_list) == 2:
                    video1 = parts_list[0]
                    video2 = parts_list[1]
                    
                    textfile_content = f"file '{video1}'\nfile '{video2}'"

                    #create the file with content
                    with open('dummy.txt', 'w') as textfile:
                        textfile.write(textfile_content)

                    #define parameters for concat function
                    textfile_name = "dummy.txt"

                    output_file = f"{video_name.split('.')[0]}_concatted{i}.mp4"
                    
                    parts_list.clear()
                    parts_list.append(output_file)
                    print(textfile_content, "output:", output_file)
                    concatenate_videos(textfile_name,output_file)                
                    remove_parts(video_name)
                    
        except Exception as e:
            print("skip to next video because:",e)

except Exception as error:
    print("An error occured:", error)   

skipped this Normalverteilung.mp4 because it is already finished
skipped this ELR_Geradengleichung.mp4 because it is already finished
skipped this ELR_Geradengleichung_Einleitung.mp4 because it is already finished
skipped this Normalverteilung2.mp4 because it is already finished
Following file does not exist: nan


# Handle the video outputs

In [83]:
rename_part1_videos()

In [84]:
# create a list with videonames of the youngest version that is not in trash list
files = []

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    youngest_creation_time = None
    youngest_video = None 
    pattern = re.compile(rf'^{re.escape(original_prefix)}_part\d+.*$')


    for f in os.listdir():     
        if f.endswith(".mp4") and f not in trash_list and not pattern.match(f):
            if "_" in f:
                part_prefix = remove_end_in_name(f)
            else:
                part_prefix = f.split('.')[0]
            
            if part_prefix == original_prefix:
                creation_time = get_creation_time(f)
                if youngest_creation_time is None or creation_time > youngest_creation_time:
                    youngest_creation_time = creation_time
                    youngest_video = f     
    if youngest_video is not None:
        files.append(youngest_video)    
  
print("All the youngest versions of a video:",files)

All the youngest versions of a video: ['Normalverteilung.mp4', 'ELR_Geradengleichung_Einleitung.mp4', 'Normalverteilung2.mp4']


In [85]:
# create a list of original video_names
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [86]:
#Compare the filenames with the original Videos
files_to_reaname = [element for element in files if element not in original_video_names]

print("Files that are gonna be renamed:",files_to_reaname)

Files that are gonna be renamed: []


In [87]:
# rename the videos
try:
    for file in files_to_reaname:
        new_name = f"{file.split('.')[0]}_cut_things_out_endprodukt.mp4"
        os.rename(file,new_name)
        print(new_name)
except Exception as e:
    print("Error while renaming:",e)

# Move files

In [88]:
for filename in os.listdir():
    if filename.endswith("endprodukt.mp4"):
        shutil.move(filename,path_folder_endprodukte)
    elif filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_script_output)

# Delete old endproducts

In [89]:
# create a list with videonames of the youngest version that is not in trash list
files = []

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    youngest_creation_time = None
    youngest_video = None 
    pattern = re.compile(rf'^{re.escape(original_prefix)}_(concatted\d+|standbild).*$')
    print(pattern)
    for f in os.listdir("endprodukte/"):     
    
        if pattern.match(f):
            #print(pattern, "matches", f)
            creation_time = get_creation_time("endprodukte/"+f)
            if youngest_creation_time is None or creation_time > youngest_creation_time:
                youngest_creation_time = creation_time
                youngest_video = f     
    if youngest_video is not None:
        files.append(youngest_video)    
  
print("All the youngest versions of a video:",files)

re.compile('^Normalverteilung_(concatted\\d+|standbild).*$')
re.compile('^ELR_Geradengleichung_(concatted\\d+|standbild).*$')
re.compile('^ELR_Geradengleichung_Einleitung_(concatted\\d+|standbild).*$')
re.compile('^Normalverteilung2_(concatted\\d+|standbild).*$')
re.compile('^nan_(concatted\\d+|standbild).*$')
All the youngest versions of a video: []


In [90]:
remove_list = []

for f in os.listdir("endprodukte/"):     
    if f not in files:
        remove_list.append(f)

print(remove_list)

[]


In [91]:
for i in remove_list:
    os.remove("endprodukte/" + i)

In [92]:
folder_path = "endprodukte/"

for video_name in video_schnitt_df["dateiname"]:
    original_prefix = video_name.split('.')[0]
    pattern = re.compile(rf'^{re.escape(original_prefix)}_(concatted\d+|standbild).*$')
    
    for f in os.listdir(folder_path):
        if pattern.match(f):
            new_name = folder_path + original_prefix + ".mp4"
            os.rename(folder_path + f,new_name)

# Start insert_pauses script

In [93]:
columns_to_pause = get_columns_to_pause(video_schnitt_df)

In [94]:
# get stanbilder
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "dauer" in col:
            continue
        output = f"{video_name.split('.')[0]}_stanbild{counter}.mp4"
        timestamp = video_schnitt_df[col][row]

        try:
            print(video_name, "standbild bei:", timestamp, "output:", output)
            get_standbild(video_name,output,timestamp)
        except Exception as e:
            print(e)
        
        counter += 1

Normalverteilung.mp4 standbild bei: 00:00:30 output: Normalverteilung_stanbild1.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Normalverteilung.mp4 standbild bei: 00:01:00 output: Normalverteilung_stanbild2.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Normalverteilung.mp4 standbild bei: 00:02:00 output: Normalverteilung_stanbild3.mp4


...

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

ELR_Geradengleichung.mp4 standbild bei: 00:02:00 output: ELR_Geradengleichung_stanbild2.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

ELR_Geradengleichung_Einleitung.mp4 standbild bei: 00:02:00 output: ELR_Geradengleichung_Einleitung_stanbild1.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

In [95]:
# get audios with silence for duration
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "bei" in col:
            continue
        
        output = f"{video_name.split('.')[0]}_audio{counter}.mp4"
        timestamp = video_schnitt_df[col][row]
        
        try:
            print(video_name, "silence for:", timestamp, "output:", output)
            create_silent_audio(timestamp,output)
        except Exception as e:
            print(e)

        counter += 1

Normalverteilung.mp4 silence for: 00:00:10 output: Normalverteilung_audio1.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Silent audio file Normalverteilung_audio1.mp4 created successfully.
Normalverteilung.mp4 silence for: 00:00:30 output: Normalverteilung_audio2.mp4


size=      15kB time=00:00:29.99 bitrate=   4.0kbits/s speed= 141x    
video:0kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 75.638321%
[aac @ 0x7fa927825e80] Qavg: 65536.000
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontcon

Silent audio file Normalverteilung_audio2.mp4 created successfully.
Normalverteilung.mp4 silence for: 00:01:00 output: Normalverteilung_audio3.mp4


size=      28kB time=00:00:59.98 bitrate=   3.9kbits/s speed= 129x    
video:0kB audio:17kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 70.866325%
[aac @ 0x7fc82b207f80] Qavg: 65536.000
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontco

Silent audio file Normalverteilung_audio3.mp4 created successfully.
ELR_Geradengleichung.mp4 silence for: 00:00:10 output: ELR_Geradengleichung_audio1.mp4
Silent audio file ELR_Geradengleichung_audio1.mp4 created successfully.
ELR_Geradengleichung.mp4 silence for: 00:00:30 output: ELR_Geradengleichung_audio2.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Silent audio file ELR_Geradengleichung_audio2.mp4 created successfully.
ELR_Geradengleichung_Einleitung.mp4 silence for: 00:00:30 output: ELR_Geradengleichung_Einleitung_audio1.mp4
Silent audio file ELR_Geradengleichung_Einleitung_audio1.mp4 created successfully.


size=      15kB time=00:00:29.99 bitrate=   4.0kbits/s speed= 132x    
video:0kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 75.638321%
[aac @ 0x7f84f7733100] Qavg: 65536.000


In [96]:
# merge all the files
for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
    counter = 1
    
    for idx, col in enumerate(columns_to_pause):
        if video_schnitt_df[col][row] == "nan" or "bei" in col:
            continue
        
        video_file = f"{video_name.split('.')[0]}_stanbild{counter}.mp4"
        audio_file = f"{video_name.split('.')[0]}_audio{counter}.mp4"
        output_file = f"{video_name.split('.')[0]}_pause{counter}.mp4"
        try :
            print(video_name,audio_file,output_file)
            merge_audio_and_video(video_file,audio_file,output_file)
        except Exception as e:
            print(e)
            
        counter += 1

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Normalverteilung.mp4 Normalverteilung_audio1.mp4 Normalverteilung_pause1.mp4


[libx264 @ 0x7fc794f05080] using SAR=1/1
[libx264 @ 0x7fc794f05080] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7fc794f05080] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7fc794f05080] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=6 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'Normalverteilung_pause1.mp4':
  Metadata:
    major_brand     : isom
    minor_version 

Normalverteilung.mp4 Normalverteilung_audio2.mp4 Normalverteilung_pause2.mp4
Normalverteilung.mp4 Normalverteilung_audio3.mp4 Normalverteilung_pause3.mp4


[libx264 @ 0x7fae19107000] using SAR=1/1
[libx264 @ 0x7fae19107000] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7fae19107000] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7fae19107000] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=6 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'Normalverteilung_pause2.mp4':
  Metadata:
    major_brand     : isom
    minor_version 

...

frame=    1 fps=0.0 q=29.0 Lsize=      26kB time=00:00:09.98 bitrate=  20.9kbits/s speed=85.2x    
video:19kB audio:3kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 14.598413%
[libx264 @ 0x7fb73401a8c0] frame I:1     Avg QP:26.35  size: 19258
[libx264 @ 0x7fb73401a8c0] mb I  I16..4: 17.1% 73.3%  9.6%
[libx264 @ 0x7fb73401a8c0] 8x8 transform intra:73.3%
[libx264 @ 0x7fb73401a8c0] coded y,uvDC,uvAC intra: 6.1% 0.0% 0.0%
[libx264 @ 0x7fb73401a8c0] i16 v,h,dc,p: 78% 13%  9%  0%
[libx264 @ 0x7fb73401a8c0] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 85%  8%  6%  0%  0%  0%  0%  0%  0%
[libx264 @ 0x7fb73401a8c0] i4 v,h,dc,ddl,ddr,vr,hd,vl,hu: 54% 15%  9%  3%  4%  4%  3%  4%  3%
[libx264 @ 0x7fb73401a8c0] i8c dc,h,v,p: 100%  0%  0%  0%
[libx264 @ 0x7fb73401a8c0] kb/s:4621.92
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthr

ELR_Geradengleichung.mp4 ELR_Geradengleichung_audio2.mp4 ELR_Geradengleichung_pause2.mp4


frame=    1 fps=0.0 q=29.0 Lsize=      35kB time=00:00:29.99 bitrate=   9.6kbits/s speed= 433x    
video:20kB audio:8kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 24.458492%
[libx264 @ 0x7fc988f065c0] frame I:1     Avg QP:26.66  size: 19720
[libx264 @ 0x7fc988f065c0] mb I  I16..4: 17.3% 73.1%  9.6%
[libx264 @ 0x7fc988f065c0] 8x8 transform intra:73.1%
[libx264 @ 0x7fc988f065c0] coded y,uvDC,uvAC intra: 6.1% 0.0% 0.0%
[libx264 @ 0x7fc988f065c0] i16 v,h,dc,p: 78% 13%  9%  0%
[libx264 @ 0x7fc988f065c0] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu: 81% 12%  7%  0%  0%  0%  0%  0%  0%
[libx264 @ 0x7fc988f065c0] i4 v,h,dc,ddl,ddr,vr,hd,vl,hu: 54% 15%  9%  3%  4%  4%  4%  4%  3%
[libx264 @ 0x7fc988f065c0] i8c dc,h,v,p: 100%  0%  0%  0%
[libx264 @ 0x7fc988f065c0] kb/s:4732.80
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthr

ELR_Geradengleichung_Einleitung.mp4 ELR_Geradengleichung_Einleitung_audio1.mp4 ELR_Geradengleichung_Einleitung_pause1.mp4


[libx264 @ 0x7ff2ea60a300] using SAR=1/1
[libx264 @ 0x7ff2ea60a300] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x7ff2ea60a300] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x7ff2ea60a300] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=6 lookahead_threads=1 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'ELR_Geradengleichung_Einleitung_pause1.mp4':
  Metadata:
    major_brand     : isom
   

## Handle output 

In [97]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

Folder 'endprodukte' already exists.
Folder 'script_output' already exists.


In [98]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [99]:
# move pauses
try:
    for video_name in video_schnitt_df["dateiname"]:
        original_prefix = video_name.split('.')[0]
        pattern = re.compile(rf'^{re.escape(original_prefix)}_pause\d+.*$')
        
        for f in os.listdir():
            if pattern.match(f):
                print(f, "moved to:", path_folder_endprodukte)
                shutil.move(f,path_folder_endprodukte)
            #elif f.endswith(".mp4") and f not in original_video_names:
                #os.remove(f)

except Exception as e:
    print(e, "or this file already has been removed")

Normalverteilung_pause1.mp4 moved to: endprodukte
Normalverteilung_pause2.mp4 moved to: endprodukte
Normalverteilung_pause3.mp4 moved to: endprodukte
ELR_Geradengleichung_pause2.mp4 moved to: endprodukte
ELR_Geradengleichung_pause1.mp4 moved to: endprodukte
ELR_Geradengleichung_Einleitung_pause1.mp4 moved to: endprodukte


In [100]:
# delete byproducts
try:
    for video_name in video_schnitt_df["dateiname"]:
        
        for f in os.listdir():
            if f.endswith(".mp4") and f not in original_video_names:
                os.remove(f)

except Exception as e:
    print(e, "or this file already has been removed")

## Cut videos into pieces and concat with pauses

In [101]:
# INPUT FILE NEEDS TO BE Adjusted. maybe better file handling before?
# Right now it just checks wether original video is in dir but it or sure is. 

folder_path = "endprodukte/"
versions_to_concat = {}

try:
    for row, video_name in video_schnitt_df[["dateiname"] + columns_to_pause].query("pause1_bei != 'nan'")["dateiname"].items():
        counter = 1
        columns_to_pause = get_columns_to_pause(video_schnitt_df)
        for col_num, col in enumerate(video_schnitt_df[columns_to_pause]):
            
            if "bei" in col and video_schnitt_df[col][row] != "nan":
                
                # Start bei mindestens einer Teilung
                if counter == 1:

                    # define all variables needed
                    if os.path.exists(folder_path + video_name):
                        input_file = folder_path + video_name
                    else:
                        input_file = video_name

                    # timestamp                
                    timestamp = video_schnitt_df[col][row]
                    # print("input timestamp:",timestamp)
                    timestamp = calculate_timestamp(timestamp,row)
                    # print("output timestamp:",timestamp)
                    
                    head_output = f"{video_name.split('.')[0]}_head{counter}.mp4"
                    tail_output = f"{video_name.split('.')[0]}_tail{counter}.mp4"
                    
                    print("all variables:",input_file,timestamp,head_output,tail_output)

                    #start separation
                    cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                    
                    versions_to_concat[video_name] = [head_output,f"{folder_path}{video_name.split('.')[0]}_pause{counter}.mp4",tail_output]
                    
                    counter += 1
                
                else:

                    # define alle the vars
                    input_file = f"{video_name.split('.')[0]}_tail{counter-1}.mp4"

                    # timestamp
                    timestamp1 = video_schnitt_df[columns_to_pause[col_num -2]][row]
                    timestamp1 = calculate_timestamp(timestamp1,row)

                    timestamp2 = video_schnitt_df[col][row]
                    timestamp2 = calculate_timestamp(timestamp2,row)

                    timestamp = calculate_time(timestamp2,timestamp1,kind="subtraction")


                    head_output = f"{video_name.split('.')[0]}_mitte{counter-1}.mp4"
                    tail_output = f"{video_name.split('.')[0]}_tail{counter}.mp4"
                    print("PRINT: timestamps before calculation:",timestamp1,timestamp2)
                    print("PRINT: all Variables further cut:",input_file,timestamp,head_output,tail_output)

                    cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                    os.remove(input_file)

                    versions_to_concat[video_name].pop()
                    versions_to_concat[video_name].append(head_output)
                    versions_to_concat[video_name].append(f"{folder_path}{video_name.split('.')[0]}_pause{counter}.mp4")
                    versions_to_concat[video_name].append(tail_output)

                    counter += 1
        
            
except Exception as e:
    print("Error in insert_pause part:", e)

all variables: Normalverteilung.mp4 00:00:30.000 Normalverteilung_head1.mp4 Normalverteilung_tail1.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

PRINT: timestamps before calculation: 00:00:30.000 00:01:00.000
PRINT: all Variables further cut: Normalverteilung_tail1.mp4 00:00:30.000 Normalverteilung_mitte1.mp4 Normalverteilung_tail2.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

PRINT: timestamps before calculation: 00:01:00.000 00:02:00.000
PRINT: all Variables further cut: Normalverteilung_tail2.mp4 00:01:00.000 Normalverteilung_mitte2.mp4 Normalverteilung_tail3.mp4


...

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

PRINT: timestamps before calculation: 00:01:00.000 00:02:00.000
PRINT: all Variables further cut: ELR_Geradengleichung_tail1.mp4 00:01:00.000 ELR_Geradengleichung_mitte1.mp4 ELR_Geradengleichung_tail2.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

all variables: ELR_Geradengleichung_Einleitung.mp4 00:02:00.000 ELR_Geradengleichung_Einleitung_head1.mp4 ELR_Geradengleichung_Einleitung_tail1.mp4


frame= 3602 fps=0.0 q=-1.0 Lsize=   37574kB time=00:01:59.97 bitrate=2565.5kbits/s speed= 586x    
video:34631kB audio:2812kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.348401%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig -

In [102]:
# Concat all the video snippets
try:
    for key, values in versions_to_concat.items():
        textfile_content = ""
        output = f"{key.split('.')[0]}_endprodukt.mp4"

        for value in values:
            dummy_string = f"file '{value}'\n"
            textfile_content = textfile_content + dummy_string

        # Create textfile with content
        textfile_name = "dummy.txt"
        with open(textfile_name, 'w') as textfile:
            textfile.write(textfile_content.rstrip())
            
        # Concat the videos
        print("key:",key,"ouput:",output,"textfile_content:", textfile_content)
        concatenate_videos(textfile_name, output)

except Exception as e:
    print(e)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

key: Normalverteilung.mp4 ouput: Normalverteilung_endprodukt.mp4 textfile_content: file 'Normalverteilung_head1.mp4'
file 'endprodukte/Normalverteilung_pause1.mp4'
file 'Normalverteilung_mitte1.mp4'
file 'endprodukte/Normalverteilung_pause2.mp4'
file 'Normalverteilung_mitte2.mp4'
file 'endprodukte/Normalverteilung_pause3.mp4'
file 'Normalverteilung_tail3.mp4'



Input #0, concat, from 'dummy.txt':
  Duration: N/A, start: 0.000000, bitrate: 235 kb/s
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 43 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.3.100 libx264
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 191 kb/s
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Output #0, mp4, to 'Normalverteilung_endprodukt.mp4':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], q=2-31, 43 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.3.100 libx264
  Stream #0:1(und): Audio: aac (LC) (mp4a /

key: ELR_Geradengleichung.mp4 ouput: ELR_Geradengleichung_endprodukt.mp4 textfile_content: file 'ELR_Geradengleichung_head1.mp4'
file 'endprodukte/ELR_Geradengleichung_pause1.mp4'
file 'ELR_Geradengleichung_mitte1.mp4'
file 'endprodukte/ELR_Geradengleichung_pause2.mp4'
file 'ELR_Geradengleichung_tail2.mp4'



[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fda437247c0] Auto-inserting h264_mp4toannexb bitstream filter
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fda43727680] Could not find codec parameters for stream 0 (Video: h264 (avc1 / 0x31637661), none(tv, bt709), 1920x1080, 95 kb/s): unspecified pixel format
Consider increasing the value for the 'analyzeduration' (0) and 'probesize' (5000000) options
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fda43727680] Auto-inserting h264_mp4toannexb bitstream filter
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fda43724cc0] Auto-inserting h264_mp4toannexb bitstream filter
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fda44029cc0] Could not find codec parameters for stream 0 (Video: h264 (avc1 / 0x31637661), none(tv, bt709), 1920x1080, 63 kb/s): unspecified pixel format
Consider increasing the value for the 'analyzeduration' (0) and 'probesize' (5000000) options
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fda44029cc0] Auto-inserting h264_mp4toannexb bitstream filter
frame= 5507 fps=0.0 q=-1.0 Lsize=    8785kB time=00:05:40.05 bitrate= 211.

key: ELR_Geradengleichung_Einleitung.mp4 ouput: ELR_Geradengleichung_Einleitung_endprodukt.mp4 textfile_content: file 'ELR_Geradengleichung_Einleitung_head1.mp4'
file 'endprodukte/ELR_Geradengleichung_Einleitung_pause1.mp4'
file 'ELR_Geradengleichung_Einleitung_tail1.mp4'



[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fc495f13d40] Auto-inserting h264_mp4toannexb bitstream filter
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fc495f27dc0] Auto-inserting h264_mp4toannexb bitstream filter
frame= 8893 fps=0.0 q=-1.0 Lsize=   43276kB time=00:05:30.05 bitrate=1074.1kbits/s speed= 529x    
video:35910kB audio:7040kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.760130%


## File Handling

In [103]:
# delete byproducts
try:
    for key, values in versions_to_concat.items():
        if os.path.exists(folder_path + key):
            os.remove(folder_path + key)

        for value in values:
            os.remove(value)
            
except Exception as e:
    print(e)


In [104]:
# Move endprodukte
for f in os.listdir():
    if f.endswith("endprodukt.mp4"):
        shutil.move(f,folder_path)

In [105]:
# Rename files
for f in os.listdir(folder_path):
    if f.endswith("endprodukt.mp4"):
        new_name = remove_end_in_name(f)
        new_name = new_name + ".mp4"
        os.rename(folder_path+f, folder_path+new_name)


# Start cut_and_keep script

In [132]:
#loop through videonames and then through columns to divide
folder_path = "endprodukte/"

try:
    for row, video_name in video_schnitt_df["dateiname"].items():
        columns_to_divide = get_columns_to_divide(video_schnitt_df)
        for col_num,col in enumerate(video_schnitt_df[columns_to_divide]):

            # check wether teilen colum has a value
            if video_schnitt_df[col][row] != "nan":

                # Wenn mehr als eine Teilung
                if col_num > 0:
                    continue
                    # define alle the vars
                    input_file = f"{video_name.split('.')[0]}_tail{col_num}.mp4"

                    # timestamp
                    timestamp_original = video_schnitt_df[columns_to_divide[col_num -1]][row]
                    timestamp1 = calculate_timestamp(timestamp_original,row)
                    timestamp2 = calculate_timestamp_from_pauses(timestamp_original,row)
                    new_timestamp1 = calculate_time(timestamp2, timestamp1, kind="subtraction")

                    timestamp_original = video_schnitt_df[col][row]
                    timestamp1 = calculate_timestamp(timestamp_original,row)
                    timestamp2 = calculate_timestamp_from_pauses(timestamp_original,row)
                    new_timestamp2 = calculate_time(timestamp2, timestamp1, kind="subtraction")

                    timestamp = calculate_time(new_timestamp2,new_timestamp1,kind="subtraction")


                    head_output = f"{video_name.split('.')[0]}_mitte{col_num+ 1}.mp4"
                    tail_output = f"{video_name.split('.')[0]}_tail{col_num +1}.mp4"
                    # print("PRINT: timestamps before calculation:",timestamp1,timestamp2)
                    print("PRINT: all Variables further cut:",input_file,timestamp,head_output,tail_output)

                    #cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
                    #os.remove(input_file)
                
                # Start bei mindestens einer Teilung
                else:

                    # define all variables needed
                    # input_file
                    if video_schnitt_df["standbild_bis"][row] == "nan" and video_schnitt_df["rausschneiden1_bis"][row] == "nan" and video_schnitt_df["pause1_bei"][row] == "nan":
                        input_file = video_name
                        
                    else:
                        input_file = folder_path + video_name
                    
                    # timestamp                
                    timestamp_original = video_schnitt_df[col][row]
                    # print("timestamp_original:",timestamp_original)
                    timestamp1 = calculate_timestamp(timestamp_original,row)
                    # #print("timestamp1:", timestamp1)
                    timestamp2 = calculate_timestamp_from_pauses(timestamp_original,row)
                    # print("timestamp2:",timestamp2)
                    new_timestamp = calculate_time(timestamp2, timestamp1, kind="subtraction")

                    # print("new timestamp:",new_timestamp)
                    
                    head_output = f"{video_name.split('.')[0]}_head{col_num +1}.mp4"
                    tail_output = f"{video_name.split('.')[0]}_tail{col_num +1}.mp4"
                    
                    #print("all variables:",input_file,new_timestamp,head_output,tail_output)

                    #start separation
                    #cut_in_2_pieces(input_file,timestamp,head_output,tail_output)
except Exception as e:
    print("Error in cut and keep script:", e)

endprodukte/Normalverteilung.mp4
index 6 is out of bounds for axis 0 with size 6
endprodukte/ELR_Geradengleichung.mp4
index 6 is out of bounds for axis 0 with size 6
Normalverteilung2.mp4
index 6 is out of bounds for axis 0 with size 6


In [ ]:
timestamp_original = "00:02:01"
timestamp1 = calculate_timestamp(timestamp_original,0)
timestamp2 = calculate_timestamp_from_pauses(timestamp_original,0)
new_timestamp2 = calculate_time(timestamp2, timestamp1, kind="subtraction")
print(timestamp1,timestamp2,new_timestamp2)

In [ ]:
calculate_time("00:01:00","00:01:00", kind="subtraction")

# Create folder, move and delete files

In [ ]:
path_folder_endprodukte = "endprodukte"
create_folder(path_folder_endprodukte)

path_folder_script_output = "script_output"
create_folder(path_folder_script_output)

In [ ]:
original_video_names = list()

for video_name in video_schnitt_df["dateiname"]:
    if video_name != "nan":
        original_video_names.append(video_name)

In [ ]:
for filename in os.listdir():
    if filename.endswith("mp4") and filename not in original_video_names:
        shutil.move(filename,path_folder_endprodukte)

In [ ]:
#video_schnitt_df["teilen1_bei"][row] == 

In [ ]:
folder_path = "endprodukte/"


for row, video_name in video_schnitt_df["dateiname"].items():
    original_prefix = video_name.split('.')[0]
    pattern = re.compile(rf'^{re.escape(original_prefix)}_(head\d+|mitte\d+|tail\d+).*$')
    
    try:    
        for f in os.listdir(folder_path):
            if pattern.match(f):
                print("video_name",video_name,"original_prefix:", original_prefix,"file",f)
                os.remove(folder_path + video_name)
    except Exception as e:
        print(e, "or this file already has been removed")

In [ ]:
os.remove("dummy.txt")

In [ ]:
print("Script reached its end")